In [1]:
#Import the required library and check the directory you are in
import os
import pandas as pd
import requests, time, io
from rdkit import Chem
from io import StringIO
import matplotlib.pyplot as plt    
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
print(os.getcwd())

#Don't run the code here ; its a clean file to collect the code snippets only; you will get error here since no data is uploaded

C:\Users\


In [2]:
# The function to get the molecules from pubchem; with the requirement of given substructure
#The function will save the smiles with its molecular weight and CID 

def get_molecules(query, file_name):
    prolog="https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    mydata = { 'smiles' : query }
    url = prolog + "/compound/fastsubstructure/smiles/cids/txt?Stereo=exact"
    res = requests.post(url, data=mydata)
    cids = res.text.split()
    print("# Number of CIDs: {}".format(len(cids)))
    chunk_size = 50
    if ( len(cids) % chunk_size == 0 ) :
        num_chunks = len(cids) // chunk_size
    else :
        num_chunks = len(cids) // chunk_size + 1
    cids_list=cids
    print("# Number of chunks:{}".format(num_chunks ))
    csv = ""   #sets a variable called csv to save the comma separated output
    for i in range(num_chunks) :
        print(i, end=" ")
        idx1 = chunk_size * i
        idx2 = chunk_size * (i + 1)
        cids_str = ",".join([ str(x) for x in cids_list[idx1:idx2] ]) # build pug input for chunks of data
        url = prolog + "/compound/cid/" + cids_str + "/property/MolecularWeight,CanonicalSMILES/csv"
        res = requests.get(url)
        if ( i == 0 ) : # if this is the first request, store result in empty csv variable
            csv = res.text 
        else :          # if this is a subsequent request, add the request to the csv variable adding a new line between chunks
            csv = csv + "\n".join(res.text.split()[1:]) + "\n" 
        time.sleep(0.2)
    csv_file = StringIO(csv)
    df_raw = pd.read_csv(csv_file, sep=",")
    return df_raw.to_csv('{}.csv'.format(file_name), index=False)

In [ ]:
#a for loop to get all the mol file of the smiles from the dataframe
mols=[Chem.MolFromSmiles(i) for i in df['CanonicalSMILES']]

In [ ]:
#This is one of the most useful function 
# It removes all the unnecessary fragments from the obtained smile
#Couple of plus point here is that it removes the metallic fragment as generally
#they are combined in the smiles with ('.')

def get_largest_fragment(smile):
    molecules=smile.split('.')
    molecules.sort(key=len)
    return(molecules[-1])

#Now apply this function to your dataframe to get the smiles with only the largest fragment

In [ ]:
# A function to find the number of atoms/substructure in a smile
#you need a dataframe with a column containg the mols and column_name 'mol'

def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

number_of_atoms(['C','O', 'N', 'P'], df)

In [ ]:
#common filters to get molecules with desired constraint like
df=df[df['num_of_N_atoms']==3]
df=df[df['num_of_O_atoms']<=4]
df=df[df['num_of_P_atoms']==0]
...

In [ ]:
#one filter to get the low molecular weight compounds
df=df[df['MolecularWeight']<=700]

In [ ]:
#Don't forget to remove the duplicates smiles; pandas are very useful to do that 
#for example

df.drop_duplicates(subset='CanonicalSMILES', keep='first', inplace=True)